PRIMEIRA CAMADA

In [0]:
file_location_clientes = "dbfs:/FileStore/seminovos/database/clientes_raw.csv"
clientes_raw = spark.read.csv(file_location_clientes, header=True, inferSchema=False)

SEGUNDA CAMADA

In [0]:
clientes_raw.write.format("delta").mode("overwrite").save("dbfs:/FileStore/seminovos/database/mnt/delta/stg_clientes")

In [0]:
file_location_clientes = "dbfs:/FileStore/seminovos/database/mnt/delta/stg_clientes"
stg_clientes = spark.read.format("delta").load(file_location_clientes)
stg_clientes.show(5)

+-----------------+--------------+--------------------+----------------+
|             Nome|           CPF|               Email|        Telefone|
+-----------------+--------------+--------------------+----------------+
| Ana Julia Moraes|273.498.651-55|helena71@example.com|+55 84 0413-5930|
|     Raquel Jesus|352.408.796-56|araujomariana@exa...|+55 11 1666 6664|
|  Evelyn Caldeira|541.730.986-93|isabel15@example.com|   0500-557-3583|
|    Alexia Novaes|968.415.302-33|lauramonteiro@exa...|+55 31 6306-4522|
|Sr. Samuel da Paz|908.324.576-47|  crocha@example.com| (031) 3757-7794|
+-----------------+--------------+--------------------+----------------+
only showing top 5 rows



In [0]:
cpf_startwith_5 = stg_clientes.filter(stg_clientes.CPF.startswith("5"))
cpf_startwith_5.show(5)

+------------------+--------------+--------------------+---------------+
|              Nome|           CPF|               Email|       Telefone|
+------------------+--------------+--------------------+---------------+
|   Evelyn Caldeira|541.730.986-93|isabel15@example.com|  0500-557-3583|
|Mariane Nascimento|536.802.419-33|ryanribeiro@examp...|(081) 7530 5447|
|Caroline das Neves|520.169.874-30|emanuellyjesus@ex...|(061) 7491 1359|
| Emanuelly da Mata|536.182.074-17|sophieviana@examp...|(021) 0419 0762|
|      Emilly Jesus|571.490.283-14|raquel48@example.org|(061) 9477-8617|
+------------------+--------------+--------------------+---------------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import col, count

filtered_clientes = stg_clientes.filter(
    (col("Nome").startswith("A")) |
    (col("Nome").startswith("B")) &
    (col("CPF").startswith("5"))
)

filtered_clientes.show(5)


+----------------+--------------+--------------------+-------------------+
|            Nome|           CPF|               Email|           Telefone|
+----------------+--------------+--------------------+-------------------+
|Ana Julia Moraes|273.498.651-55|helena71@example.com|   +55 84 0413-5930|
|   Alexia Novaes|968.415.302-33|lauramonteiro@exa...|   +55 31 6306-4522|
|  Alana da Rocha|132.490.865-33|  levi50@example.net|+55 (031) 7553 0116|
| Alice Gonçalves|264.850.139-89| tda-luz@example.com|      0300 186 2162|
| Agatha Ferreira|603.845.972-38|marcos-vinicius64...|       81 9248-9496|
+----------------+--------------+--------------------+-------------------+
only showing top 5 rows



Aplicando tratamento no camp 'CPF'

In [0]:
count_cpfs = stg_clientes.groupBy("CPF").agg(count("cpf").alias("quantidade_cpf"))
count_cpfs.show()

+--------------+--------------+
|           CPF|quantidade_cpf|
+--------------+--------------+
|249.571.380-79|             1|
|734.128.650-44|             1|
|037.286.154-71|             1|
|541.708.963-00|             1|
|309.541.768-39|             1|
|365.729.480-56|             1|
|743.619.852-19|             1|
|509.213.687-12|             1|
|237.806.519-12|             1|
|267.194.358-09|             1|
|198.053.276-12|             1|
|173.940.256-16|             1|
|089.126.734-40|             1|
|379.285.140-79|             1|
|708.493.152-04|             1|
|085.143.296-42|             1|
|912.306.784-50|             1|
|360.589.271-02|             1|
|310.296.574-16|             1|
|821.903.456-42|             1|
+--------------+--------------+
only showing top 20 rows



In [0]:

cpfs_duplicados = count_cpfs.filter(col("quantidade_cpf")>1)
cpfs_duplicados.show()

+--------------+--------------+
|           CPF|quantidade_cpf|
+--------------+--------------+
|637.824.051-53|             2|
+--------------+--------------+



In [0]:
nomes_cpfs_duplicados = stg_clientes.filter(col("CPF")== "637.824.051-53")
nomes_cpfs_duplicados.show(truncate=False)

+-------------------+--------------+--------------------------+-------------------+
|Nome               |CPF           |Email                     |Telefone           |
+-------------------+--------------+--------------------------+-------------------+
|Luiz Miguel Pires  |637.824.051-53|bcastro@example.com       |+55 (041) 8718-1716|
|Murilo da Conceição|637.824.051-53|martinsleandro@example.com|+55 61 2216 2442   |
+-------------------+--------------+--------------------------+-------------------+



In [0]:
from pyspark.sql.functions import col, when
cliente_atualizado = stg_clientes.withColumn(
    "CPF",
    when(
    (col("Nome")== "Luiz Miguel Pires") & (col("CPF")== "637.824.051-53"),
    "637.824.059-59"
    ).otherwise
    (col("CPF"))
)
cliente_atualizado.show(truncate=False)


+-----------------------+--------------+-----------------------------+-------------------+
|Nome                   |CPF           |Email                        |Telefone           |
+-----------------------+--------------+-----------------------------+-------------------+
|Ana Julia Moraes       |273.498.651-55|helena71@example.com         |+55 84 0413-5930   |
|Raquel Jesus           |352.408.796-56|araujomariana@example.net    |+55 11 1666 6664   |
|Evelyn Caldeira        |541.730.986-93|isabel15@example.com         |0500-557-3583      |
|Alexia Novaes          |968.415.302-33|lauramonteiro@example.net    |+55 31 6306-4522   |
|Sr. Samuel da Paz      |908.324.576-47|crocha@example.com           |(031) 3757-7794    |
|Alana da Rocha         |132.490.865-33|levi50@example.net           |+55 (031) 7553 0116|
|Eduardo da Rocha       |490.713.682-03|tda-mata@example.net         |+55 51 1723 7105   |
|Mariane Nascimento     |536.802.419-33|ryanribeiro@example.org      |(081) 7530 5447    |

In [0]:
cpf_atualizado = cliente_atualizado.filter(
    (col("Nome") == "Luiz Miguel Pires") & (col("CPF") == "637.824.059-59")
).select("CPF").collect()

if cpf_atualizado:
    print(f"CPF atualizado para {cpf_atualizado[0]['CPF']}")
else:
    print("O CPF não foi atualizado.")


CPF atualizado para 637.824.059-59


In [0]:
file_location_clientes = "dbfs:/FileStore/seminovos/database/mnt/delta/stg_clientes"
stg_clientes = spark.read.format("delta").load(file_location_clientes)
stg_clientes_telefone_limpo.write.format("delta").mode("overwrite").save("dbfs:/FileStore/seminovos/database/mnt/delta/stg_clientes")

In [0]:
from pyspark.sql.functions import col
clientes_atualizados = stg_clientes.filter(
    (col("Nome").startswith("Luiz Miguel Pires")) |
    (col("Nome").startswith("Murilo da Conceição"))
)
    
clientes_atualizados.show()

+-------------------+--------------+--------------------+-------------+
|               Nome|           CPF|               Email|     Telefone|
+-------------------+--------------+--------------------+-------------+
|  Luiz Miguel Pires|637.824.059-59| bcastro@example.com|5504187181716|
|Murilo da Conceição|637.824.051-53|martinsleandro@ex...| 556122162442|
+-------------------+--------------+--------------------+-------------+



**Aplicando transformações no campo 'Telefone':**

In [0]:
from pyspark.sql.functions import regexp_replace

stg_clientes_telefone_limpo = stg_clientes.withColumn(
    "Telefone",
    regexp_replace(col("Telefone"), "[^0-9]", "")
)
stg_clientes_telefone_limpo.show(5)

+-----------------+--------------+--------------------+------------+
|             Nome|           CPF|               Email|    Telefone|
+-----------------+--------------+--------------------+------------+
| Ana Julia Moraes|273.498.651-55|helena71@example.com|558404135930|
|     Raquel Jesus|352.408.796-56|araujomariana@exa...|551116666664|
|  Evelyn Caldeira|541.730.986-93|isabel15@example.com| 05005573583|
|    Alexia Novaes|968.415.302-33|lauramonteiro@exa...|553163064522|
|Sr. Samuel da Paz|908.324.576-47|  crocha@example.com| 03137577794|
+-----------------+--------------+--------------------+------------+
only showing top 5 rows



In [0]:
file_location_clientes = "dbfs:/FileStore/seminovos/database/mnt/delta/stg_clientes"
stg_clientes = spark.read.format("delta").load(file_location_clientes)
stg_clientes_telefone_limpo.write.format("delta").mode("overwrite").save("dbfs:/FileStore/seminovos/database/mnt/delta/stg_clientes")

In [0]:
from pyspark.sql.functions import length, col
comprimento_telefone = stg_clientes.withColumn("Telefone_length", length(col("Telefone")))
comprimento_telefone.show(5)


+-----------------+--------------+--------------------+------------+---------------+
|             Nome|           CPF|               Email|    Telefone|Telefone_length|
+-----------------+--------------+--------------------+------------+---------------+
| Ana Julia Moraes|273.498.651-55|helena71@example.com|558404135930|             12|
|     Raquel Jesus|352.408.796-56|araujomariana@exa...|551116666664|             12|
|  Evelyn Caldeira|541.730.986-93|isabel15@example.com| 05005573583|             11|
|    Alexia Novaes|968.415.302-33|lauramonteiro@exa...|553163064522|             12|
|Sr. Samuel da Paz|908.324.576-47|  crocha@example.com| 03137577794|             11|
+-----------------+--------------+--------------------+------------+---------------+
only showing top 5 rows



In [0]:
agrupar_comprimento_telefone = comprimento_telefone.groupBy("Telefone_Length").count().filter(
    col("Telefone_Length").isin(10, 11, 12, 13,14,15,16)
)
agrupar_comprimento_telefone.show(truncate=False)

+---------------+-----+
|Telefone_Length|count|
+---------------+-----+
|12             |229  |
|13             |229  |
|10             |237  |
|11             |305  |
+---------------+-----+



In [0]:
telefone_11_digitos = stg_clientes.withColumn(
    "Telefone",
    when(
        col("Telefone").rlike("^[0-9]{11}$"), 
        concat(
            lit("("),
            substring(col("Telefone"), 1, 2),  
            lit(") "),
            substring(col("Telefone"), 3, 4), 
            lit("-"),
            substring(col("Telefone"), 7, 4) 
        )
    ).otherwise(col("Telefone")) 
)
telefone_11_digitos.show(10)

+------------------+--------------+--------------------+--------------+
|              Nome|           CPF|               Email|      Telefone|
+------------------+--------------+--------------------+--------------+
|  Ana Julia Moraes|273.498.651-55|helena71@example.com|  558404135930|
|      Raquel Jesus|352.408.796-56|araujomariana@exa...|  551116666664|
|   Evelyn Caldeira|541.730.986-93|isabel15@example.com|(05) 0055-7358|
|     Alexia Novaes|968.415.302-33|lauramonteiro@exa...|  553163064522|
| Sr. Samuel da Paz|908.324.576-47|  crocha@example.com|(03) 1375-7779|
|    Alana da Rocha|132.490.865-33|  levi50@example.net| 5503175530116|
|  Eduardo da Rocha|490.713.682-03|tda-mata@example.net|  555117237105|
|Mariane Nascimento|536.802.419-33|ryanribeiro@examp...|(08) 1753-0544|
|       Raul Santos|014.958.362-15|maria-luiza92@exa...|(09) 0057-9598|
|Caroline das Neves|520.169.874-30|emanuellyjesus@ex...|(06) 1749-1135|
+------------------+--------------+--------------------+--------

In [0]:
telefone_11_digitos.write.format("delta").mode("overwrite").save("dbfs:/FileStore/seminovos/database/mnt/delta/stg_clientes")

In [0]:
telefone_12_digitos = stg_clientes.withColumn(
    "Telefone",
    when(
        col("Telefone").rlike("^[0-9]{12}$"), 
        concat(
            lit("("),
            substring(col("Telefone"), 1, 2),  
            lit(") "),
            substring(col("Telefone"), 3, 4), 
            lit("-"),
            substring(col("Telefone"), 7, 4) 
        )
    ).otherwise(col("Telefone")) 
)
telefone_12_digitos.show(10)

+------------------+--------------+--------------------+--------------+
|              Nome|           CPF|               Email|      Telefone|
+------------------+--------------+--------------------+--------------+
|  Ana Julia Moraes|273.498.651-55|helena71@example.com|(55) 8404-1359|
|      Raquel Jesus|352.408.796-56|araujomariana@exa...|(55) 1116-6666|
|   Evelyn Caldeira|541.730.986-93|isabel15@example.com|(05) 0055-7358|
|     Alexia Novaes|968.415.302-33|lauramonteiro@exa...|(55) 3163-0645|
| Sr. Samuel da Paz|908.324.576-47|  crocha@example.com|(03) 1375-7779|
|    Alana da Rocha|132.490.865-33|  levi50@example.net| 5503175530116|
|  Eduardo da Rocha|490.713.682-03|tda-mata@example.net|(55) 5117-2371|
|Mariane Nascimento|536.802.419-33|ryanribeiro@examp...|(08) 1753-0544|
|       Raul Santos|014.958.362-15|maria-luiza92@exa...|(09) 0057-9598|
|Caroline das Neves|520.169.874-30|emanuellyjesus@ex...|(06) 1749-1135|
+------------------+--------------+--------------------+--------

In [0]:
telefone_12_digitos.write.format("delta").mode("overwrite").save("dbfs:/FileStore/seminovos/database/mnt/delta/stg_clientes")

In [0]:
telefone_13_digitos = stg_clientes.withColumn(
    "Telefone",
    when(
        col("Telefone").rlike("^[0-9]{13}$"), 
        concat(
            lit("("),
            substring(col("Telefone"), 1, 2),  
            lit(") "),
            substring(col("Telefone"), 3, 4), 
            lit("-"),
            substring(col("Telefone"), 7, 4) 
        )
    ).otherwise(col("Telefone")) 
)
telefone_13_digitos.show(10)

+------------------+--------------+--------------------+--------------+
|              Nome|           CPF|               Email|      Telefone|
+------------------+--------------+--------------------+--------------+
|  Ana Julia Moraes|273.498.651-55|helena71@example.com|(55) 8404-1359|
|      Raquel Jesus|352.408.796-56|araujomariana@exa...|(55) 1116-6666|
|   Evelyn Caldeira|541.730.986-93|isabel15@example.com|(05) 0055-7358|
|     Alexia Novaes|968.415.302-33|lauramonteiro@exa...|(55) 3163-0645|
| Sr. Samuel da Paz|908.324.576-47|  crocha@example.com|(03) 1375-7779|
|    Alana da Rocha|132.490.865-33|  levi50@example.net|(55) 0317-5530|
|  Eduardo da Rocha|490.713.682-03|tda-mata@example.net|(55) 5117-2371|
|Mariane Nascimento|536.802.419-33|ryanribeiro@examp...|(08) 1753-0544|
|       Raul Santos|014.958.362-15|maria-luiza92@exa...|(09) 0057-9598|
|Caroline das Neves|520.169.874-30|emanuellyjesus@ex...|(06) 1749-1135|
+------------------+--------------+--------------------+--------

In [0]:
telefone_13_digitos.write.format("delta").mode("overwrite").save("dbfs:/FileStore/seminovos/database/mnt/delta/stg_clientes")

**Aplicando tratamento no campo email**

In [0]:
from pyspark.sql.functions import col, concat_ws, lower, regexp_replace, split, lit

cliente_email_padronizado = stg_clientes.withColumn(
    "Email",
    concat_ws(
        "",
        lower(
            regexp_replace(
                split(col("Nome"), " ")[0],  
                "[^a-zA-Z]",                  
                ""
            )
        ),
        lit("@exemplo.net")              
    )
)
cliente_email_padronizado.show(truncate=False)


+-----------------------+--------------+---------------------+--------------+
|Nome                   |CPF           |Email                |Telefone      |
+-----------------------+--------------+---------------------+--------------+
|Ana Julia Moraes       |273.498.651-55|ana@exemplo.net      |(55) 8404-1359|
|Raquel Jesus           |352.408.796-56|raquel@exemplo.net   |(55) 1116-6666|
|Evelyn Caldeira        |541.730.986-93|evelyn@exemplo.net   |(05) 0055-7358|
|Alexia Novaes          |968.415.302-33|alexia@exemplo.net   |(55) 3163-0645|
|Sr. Samuel da Paz      |908.324.576-47|sr@exemplo.net       |(03) 1375-7779|
|Alana da Rocha         |132.490.865-33|alana@exemplo.net    |(55) 0317-5530|
|Eduardo da Rocha       |490.713.682-03|eduardo@exemplo.net  |(55) 5117-2371|
|Mariane Nascimento     |536.802.419-33|mariane@exemplo.net  |(08) 1753-0544|
|Raul Santos            |014.958.362-15|raul@exemplo.net     |(09) 0057-9598|
|Caroline das Neves     |520.169.874-30|caroline@exemplo.net |(0

In [0]:
from pyspark.sql.functions import col

prefixos = ['Sr.', 'Dr.', 'Sra.', 'Dra.']

contagem_prefixos = stg_clientes.filter(
    col('Nome').rlike(f"^({'|'.join(prefixos)})")
).count()

print(f"Total de linhas com prefixos {prefixos}: {contagem_prefixos}")


Total de linhas com prefixos ['Sr.', 'Dr.', 'Sra.', 'Dra.']: 164


In [0]:
clientes_sem_prefixos = stg_clientes.filter(
    ~col('Nome').rlike(f"^({'|'.join(prefixos)})")
)

from pyspark.sql.functions import concat_ws, lower

clientes_email_normalizados = clientes_sem_prefixos.withColumn(
    "Email",
    concat_ws(
        "@", 
        lower(split(col("Nome"), " ")[0]), 
        lit("exemplo.net")
    )
)

clientes_email_normalizados.show(truncate=False)


+--------------------+--------------+---------------------+--------------+
|Nome                |CPF           |Email                |Telefone      |
+--------------------+--------------+---------------------+--------------+
|Ana Julia Moraes    |273.498.651-55|ana@exemplo.net      |(55) 8404-1359|
|Raquel Jesus        |352.408.796-56|raquel@exemplo.net   |(55) 1116-6666|
|Evelyn Caldeira     |541.730.986-93|evelyn@exemplo.net   |(05) 0055-7358|
|Alexia Novaes       |968.415.302-33|alexia@exemplo.net   |(55) 3163-0645|
|Alana da Rocha      |132.490.865-33|alana@exemplo.net    |(55) 0317-5530|
|Eduardo da Rocha    |490.713.682-03|eduardo@exemplo.net  |(55) 5117-2371|
|Mariane Nascimento  |536.802.419-33|mariane@exemplo.net  |(08) 1753-0544|
|Raul Santos         |014.958.362-15|raul@exemplo.net     |(09) 0057-9598|
|Caroline das Neves  |520.169.874-30|caroline@exemplo.net |(06) 1749-1135|
|Marcela Silveira    |245.068.713-08|marcela@exemplo.net  |(55) 0716-3610|
|Maria Pires         |376

In [0]:
clientes_email_normalizados.write.format("delta").mode("overwrite").save("dbfs:/FileStore/seminovos/database/mnt/delta/stg_clientes")

In [0]:
display(stg_clientes)

Nome,CPF,Email,Telefone
Ana Julia Moraes,273.498.651-55,ana@exemplo.net,(55) 8404-1359
Raquel Jesus,352.408.796-56,raquel@exemplo.net,(55) 1116-6666
Evelyn Caldeira,541.730.986-93,evelyn@exemplo.net,(05) 0055-7358
Alexia Novaes,968.415.302-33,alexia@exemplo.net,(55) 3163-0645
Alana da Rocha,132.490.865-33,alana@exemplo.net,(55) 0317-5530
Eduardo da Rocha,490.713.682-03,eduardo@exemplo.net,(55) 5117-2371
Mariane Nascimento,536.802.419-33,mariane@exemplo.net,(08) 1753-0544
Raul Santos,014.958.362-15,raul@exemplo.net,(09) 0057-9598
Caroline das Neves,520.169.874-30,caroline@exemplo.net,(06) 1749-1135
Marcela Silveira,245.068.713-08,marcela@exemplo.net,(55) 0716-3610


In [0]:
from delta.tables import DeltaTable
delta_table = DeltaTable.forPath(spark, "dbfs:/FileStore/seminovos/database/mnt/delta/stg_clientes")
historico_df = delta_table.history()

display(historico_df)

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
12,2024-08-17T18:30:46.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,11,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 836, numOutputBytes -> 30122)",null,Databricks-Runtime/12.2.x-scala2.12
11,2024-08-17T17:05:21.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,10,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 42933)",null,Databricks-Runtime/12.2.x-scala2.12
10,2024-08-17T17:04:32.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,9,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 43399)",null,Databricks-Runtime/12.2.x-scala2.12
9,2024-08-17T17:03:27.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,8,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 43427)",null,Databricks-Runtime/12.2.x-scala2.12
8,2024-08-17T17:02:23.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,7,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 43312)",null,Databricks-Runtime/12.2.x-scala2.12
7,2024-08-17T17:01:04.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,6,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 42759)",null,Databricks-Runtime/12.2.x-scala2.12
6,2024-08-17T16:51:58.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,5,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 42759)",null,Databricks-Runtime/12.2.x-scala2.12
5,2024-08-17T16:51:27.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,4,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 44367)",null,Databricks-Runtime/12.2.x-scala2.12
4,2024-08-17T15:22:36.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,3,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 44367)",null,Databricks-Runtime/12.2.x-scala2.12
3,2024-08-17T15:20:37.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,2,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 44363)",null,Databricks-Runtime/12.2.x-scala2.12


In [0]:
num_versoes = historico_df.count()
print(f"Total de versões disponíveis: {num_versoes}")

Total de versões disponíveis: 12


In [0]:
schema_df = delta_table.toDF().printSchema()


root
 |-- Nome: string (nullable = true)
 |-- CPF: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Telefone: string (nullable = true)



In [0]:
table_info = delta_table.toDF().describe()
table_info.show()


+-------+---------------+--------------+--------------------+--------------+
|summary|           Nome|           CPF|               Email|      Telefone|
+-------+---------------+--------------+--------------------+--------------+
|  count|           1000|          1000|                1000|          1000|
|   mean|           null|          null|                null|          null|
| stddev|           null|          null|                null|          null|
|    min|Agatha Ferreira|012.365.897-77|aalmeida@example.net|(01) 1067-9593|
|    max|    Yuri Vieira|984.562.017-58|  zramos@example.net|(84) 9771-5271|
+-------+---------------+--------------+--------------------+--------------+



Selecionar uma Versão com Base no Timestamp

In [0]:
from delta.tables import DeltaTable
delta_table_path = "dbfs:/FileStore/seminovos/database/mnt/delta/stg_clientes"

stg_clientes = DeltaTable.forPath(spark, delta_table_path)
historico_df = stg_clientes.history() 
display(historico_df)

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
12,2024-08-17T18:30:46.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,11,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 836, numOutputBytes -> 30122)",null,Databricks-Runtime/12.2.x-scala2.12
11,2024-08-17T17:05:21.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,10,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 42933)",null,Databricks-Runtime/12.2.x-scala2.12
10,2024-08-17T17:04:32.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,9,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 43399)",null,Databricks-Runtime/12.2.x-scala2.12
9,2024-08-17T17:03:27.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,8,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 43427)",null,Databricks-Runtime/12.2.x-scala2.12
8,2024-08-17T17:02:23.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,7,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 43312)",null,Databricks-Runtime/12.2.x-scala2.12
7,2024-08-17T17:01:04.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,6,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 42759)",null,Databricks-Runtime/12.2.x-scala2.12
6,2024-08-17T16:51:58.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,5,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 42759)",null,Databricks-Runtime/12.2.x-scala2.12
5,2024-08-17T16:51:27.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,4,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 44367)",null,Databricks-Runtime/12.2.x-scala2.12
4,2024-08-17T15:22:36.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,3,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 44367)",null,Databricks-Runtime/12.2.x-scala2.12
3,2024-08-17T15:20:37.000+0000,8647403558458507,regimaria015@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1704919530911047),0817-131508-gfn0l2g0,2,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 44363)",null,Databricks-Runtime/12.2.x-scala2.12


In [0]:
from pyspark.sql.functions import col
timestamp_objetivo = "2024-08-17T16:51:58.000+00:00"

version_df = history_df.filter(col("timestamp") == timestamp_objetivo).select("version")
version = version_df.collect()[0][0]

print(f"Versão correspondente ao timestamp {timestamp_objetivo}: {version}")

Versão correspondente ao timestamp 2024-08-17T16:51:58.000+00:00: 6


Time travel

In [0]:
version = 6
df_time_travel = spark.read.format("delta").option("versionAsOf", version).load("dbfs:/FileStore/seminovos/database/mnt/delta/stg_clientes")
df_time_travel.show(truncate=False)

+-----------------------+--------------+-----------------------------+-------------+
|Nome                   |CPF           |Email                        |Telefone     |
+-----------------------+--------------+-----------------------------+-------------+
|Ana Julia Moraes       |273.498.651-55|helena71@example.com         |558404135930 |
|Raquel Jesus           |352.408.796-56|araujomariana@example.net    |551116666664 |
|Evelyn Caldeira        |541.730.986-93|isabel15@example.com         |05005573583  |
|Alexia Novaes          |968.415.302-33|lauramonteiro@example.net    |553163064522 |
|Sr. Samuel da Paz      |908.324.576-47|crocha@example.com           |03137577794  |
|Alana da Rocha         |132.490.865-33|levi50@example.net           |5503175530116|
|Eduardo da Rocha       |490.713.682-03|tda-mata@example.net         |555117237105 |
|Mariane Nascimento     |536.802.419-33|ryanribeiro@example.org      |08175305447  |
|Raul Santos            |014.958.362-15|maria-luiza92@example.com

TERCEIRA CAMADA

In [0]:
file_location_clientes = "dbfs:/FileStore/seminovos/database/mnt/delta/stg_clientes"
delta_clientes = spark.read.format("delta").load(file_location_clientes)
delta_clientes.write.format("delta").mode("overwrite").save("dbfs:/FileStore/seminovos/database/mnt/delta/delta_clientes")

In [0]:
display(delta_clientes)

Nome,CPF,Email,Telefone
Ana Julia Moraes,273.498.651-55,ana@exemplo.net,(55) 8404-1359
Raquel Jesus,352.408.796-56,raquel@exemplo.net,(55) 1116-6666
Evelyn Caldeira,541.730.986-93,evelyn@exemplo.net,(05) 0055-7358
Alexia Novaes,968.415.302-33,alexia@exemplo.net,(55) 3163-0645
Alana da Rocha,132.490.865-33,alana@exemplo.net,(55) 0317-5530
Eduardo da Rocha,490.713.682-03,eduardo@exemplo.net,(55) 5117-2371
Mariane Nascimento,536.802.419-33,mariane@exemplo.net,(08) 1753-0544
Raul Santos,014.958.362-15,raul@exemplo.net,(09) 0057-9598
Caroline das Neves,520.169.874-30,caroline@exemplo.net,(06) 1749-1135
Marcela Silveira,245.068.713-08,marcela@exemplo.net,(55) 0716-3610
